In [ ]:
'''
    Objective: Create a battleship algorithm to 1. Hunt and 2. Target ships that minizes the number of expected moves to # WARNING:

    Game Rules:
        Objective: Sink all the other players ships
        Board Size = 10X10
        Typical Ships:
            1. Aircraft Carrier = 5
            2. Battleship = 4
            3. Submarine = 3
            4. Crusier = 3
            5. Destroyer = 2
        Ship Rules:
            1. Ships cannot overlap, but can touch
        Turn Rules:
            1. Player takes a shot
                1. Player Hits a ship
                2. Player Misses a ship
            2. If a player sinks a ship they are informed what ship was sunk

    Algorithm Details:
        Objective: Minimize the number of turns it takes to win.
        Hunt State: Target a square with the highest liklihood to hit a ship.
        Target State: Once you hit a ship, target the square wiht the highest liklihood to be a continuation of the ship.

'''

# Class and Functions to Setup the Game

In [ ]:
'Necessary libraries'
from random import randint
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statistics
import pandas as pd
import os
import time
%matplotlib inline

'Create a ship object'
class Ship:
    """Ship object"""
    def __init__(self, location, length, name, orientation,board):
        self.length = length
        self.name = name
        self.board = board

        if orientation == 'horizontal':
            if location['row'] in range(row_size):
                self.coordinates = []
                for index in range(length):
                    if location['col'] + index in range(col_size):
                        self.coordinates.append({'row': location['row'], 'col': location['col'] + index})
                    else:
                        raise IndexError('Column is out of range')
        elif orientation == 'vertical':
            if location['col'] in range(col_size):
                self.coordinates = []
                for index in range(length):
                    if location['row'] + index in range(col_size):
                        self.coordinates.append({'row': location['row'] + index, 'col': location['col']})
                    else:
                        raise IndexError('Row is out of range')
            else:
                raise IndexError('Column is out of range')
        
        if self.filled(self.board):
            raise IndexError('A ship already occupies that space.')
        else:
            self.fillBoard(self.board)
        
    def filled(self,board):
        for coords in self.coordinates:
            if board[coords['row']][coords['col']] == 1:
                return True
            elif board[coords['row']][coords['col']] == 'M': #Missed
                return True
            elif board[coords['row']][coords['col']] == 'S': #Sunk
                return True
        return False

    def fillBoard(self,board):
        for coords in self.coordinates:
            board[coords['row']][coords['col']] = 1

    def contains(self, location):
        for coords in self.coordinates:
            if coords == location:
                return True
        return False

    def destroyed(self,guess_board):
        #Convert our guess board to just integers on hit (X). Sunk (S) can be 0 since ships can't overlap. 
        guess_board_int = np.array([[1 if element == 'X' else (1 if not isinstance(element, int) else element) for element in row] for row in guess_board])
        #Check to see if the coordinates are in the ship. 
        for coords in self.coordinates:
            if guess_board_int[coords['row']][coords['col']] == 0:
                return False
        #If sunk rename X to S for sunk
        for guess_coords in self.coordinates:
            guess_board[guess_coords['row']][guess_coords['col']] = 'S'
        return True
 
    
###Inital Game Settings
#grid variables
row_size = 10
col_size = 10
#ship details
ships = [('Aircraft Carrier',5),('Battleship',4),('Submarine',3),('Cruiser',3),('Destroyer',2)]
max_turns = 100
#How many simulations in the hunt and target
algo_sim = 1000

#We need have a function that determines the valid ship locations
def search_locations(size, orientation,board,row_size,col_size):
    locations = []
    
    if orientation != 'horizontal' and orientation != 'vertical':
        raise ValueError('Orientation is not valid')
        
    if orientation == 'horizontal':
        if size <= col_size:
            for r in range(row_size):
                for c in range(col_size - size + 1):
                    if 1 not in board[r][c:c+size]:
                        locations.append({'row':r, 'col': c})
    elif orientation == 'vertical':
        if size <= row_size:
            for c in range(col_size):
                for r in range(row_size - size + 1):
                    if 1 not in [board[i][c] for i in range(r, r+size)]:
                        locations.append({'row': r, 'col': c})
                        
    if not locations:
        return 'None'
    else:
        return locations

#Function for random functions
def random_location(size, name, board, row_size, col_size):
    orientation = 'horizontal' if randint(0,1) == 0 else 'vertical'

    locations = search_locations(size, orientation, board, row_size, col_size)
    if locations == 'None':
        return 'None'
    else:
        return{'location': locations[randint(0, len(locations) - 1)], 'size': size, 'orientation': orientation, 'name': name}
    

# Functions for Optimal Strategy

In [ ]:
#Hunt algorithm to show where all the possible ship locations can be and for how best to shoot when no hits (X). 
def hunt(current_board,remaining_ships):
    #Create an empty result board. 
    result_board = [[0] * col_size for x in range(row_size)]
    #Update our guess board to convert all Miss, Hit's and Sunk to 1. 
    current_board_int = [[1 if not isinstance(element, int) else element for element in row] for row in current_board]
    for i in range(0,algo_sim):
    #Create our empty board for the Ship class.
        blank_board = [[0] * col_size for x in range(row_size)]
        hunt_ship_list = []
        placed = 0 
        while placed < len(remaining_ships):
            ship_info = random_location(remaining_ships[placed].length,remaining_ships[placed].name, np.array(current_board_int) + np.array(blank_board), row_size, col_size)
            if ship_info == 'None':
                placed += 1
                continue
            else:
                hunt_ship_list.append(Ship(ship_info['location'], ship_info['size'], ship_info['name'], ship_info['orientation'], blank_board))
            placed += 1
        result_board = np.array(result_board) + np.array(blank_board)
        #reset our base board and ship list
    
    return result_board, np.unravel_index(np.argmax(result_board, axis=None), result_board.shape)


#Similar to the hunt algorithm, but we'll instead look a smaller space around the hit (X)
def target_space(current_board,remaining_ships):
    #Create an empty result board
    mini_board = current_board.shape
    current_board = current_board.tolist()
    result_board = [[0] * mini_board[1] for x in range(mini_board[0])]
    current_board_int = [[0 if element == 'X' else (1 if not isinstance(element, int) else element) for element in row] for row in current_board]
    for i in range(0,algo_sim):
    #Create our empty board for the Ship class.
        blank_board = [[0] * mini_board[1] for x in range(mini_board[0])]
        hunt_ship_list = []
        placed = 0 
        while placed < len(remaining_ships):
            ship_info = random_location(remaining_ships[placed].length,remaining_ships[placed].name, np.array(current_board_int) + np.array(blank_board), mini_board[0], mini_board[1])
            if ship_info == 'None':
                placed += 1
                continue
            else:
                hunt_ship_list.append(Ship(ship_info['location'], ship_info['size'], ship_info['name'], ship_info['orientation'], blank_board))
            placed += 1
        result_board = np.array(result_board) + np.array(blank_board)

    return result_board

#After we create the target board we then need to find the best shot adjacent to the last hit (X).
#Leaving the function here for now, but this didn't actually work. Ran into infinite loop issues. 
# def best_target(current_board,x_indices,y_indices):
#     best_shot_coordinates = (0,0)
#     max_shot = 0
    
#     for i in [-1,1]:
#         #Check left and right of the hit first. 
#         try: 
#             if current_board[x_indices[0]+i][y_indices[0]] > max_shot:
#                 max_shot = current_board[x_indices[0]+i][y_indices[0]] 
#                 best_shot_coordinates = (x_indices[0]+i, y_indices[0])
#         except:
#             pass
            
#     for i in [-1,1]:
#         #Check north and south of the hit second. 
#         try: 
#             if current_board[x_indices[0]][y_indices[0]+i] > max_shot:
#                 max_shot = current_board[x_indices[0]][y_indices[0]+i] 
#                 best_shot_coordinates = (x_indices[0], y_indices[0]+i)
#         except:
#             pass
    
#     return best_shot_coordinates


# Function to create heatmap of best shot options with highlighted red box for best shot
def heatmap_with_red_box(matrix, shot_coordinates):
    
    # Set the size of the plot
    fig, ax = plt.subplots(figsize=(12, 12))
    
    # Create the heatmap
    sns.heatmap(matrix/algo_sim, annot=True, cmap="Blues")

    # Get the shot coordinates
    x, y = shot_coordinates

    # Add a red box around the shot coordinates
    plt.gca().add_patch(plt.Rectangle((y, x), 1, 1, linewidth=2, edgecolor='red', facecolor='none'))

    # Show the plot
    plt.show()


###Function to get the optimal shot
def get_shot(guess_board, ship_list, print_charts=False):
    
    #Create a board for the algo
    your_matrix = np.array(guess_board, dtype = object)
    #Create a special board to plot our results. 
    current_board_int = np.array([[0 if element == 'X' else (1 if not isinstance(element, int) else element) for element in row] for row in guess_board])
    
    #print out of input board and next shot coordinates with heatmap. 
    if print_charts:
        print('-'*15,'current board','-'*15)
        print_board(your_matrix)
        print('-'*15,'Next Shot','-'*15)
    
    max_ship = 0
    for i in range(0,len(ship_list)):
        if ship_list[i].length > max_ship:
            max_ship = ship_list[i].length
    print('max size ship remaining', max_ship)
    #Do we have a hit (X) on the board and therefore in Target? 
    value_to_search = 'X'
    x_indices, y_indices = np.where(your_matrix == value_to_search)
    if len(x_indices) > 0 and len(y_indices) > 0:
        #Find where there is an X on the board. 
        mode = 'target'
    #If not, we're in Hunt mode
    else:
        mode = 'hunt'
    print(mode)
    
    if mode == 'target':
        # Calculate the boundaries for the subset we want to aim at.
        min_row = max(0, np.min(x_indices) - max_ship + 1)
        max_row = min(your_matrix.shape[0], np.max(x_indices) + max_ship)
        min_col = max(0, np.min(y_indices) - max_ship + 1)
        max_col = min(your_matrix.shape[1], np.max(y_indices) + max_ship)

        # Isolate the subset matrix. 
        subset_matrix = your_matrix[min_row:max_row, min_col:max_col].copy()

        # Run target board algorithm
        target_board = target_space(subset_matrix, ship_list)

        # put the target board into the original board. 
        current_board_int[min_row:max_row, min_col:max_col] = target_board

        #current_board_int[np.ix_(x_indices, y_indices)] = 0 
        current_board_int[x_indices,y_indices] = 0

        
        #Run the algo to get the target shot coordinates. 
        #shot = best_target(current_board_int,x_indices,y_indices)
        shot = np.unravel_index(np.argmax(current_board_int, axis=None), current_board_int.shape)

        #Print coordinates and heatmap. 
        print('target mode, shot coordinates here: ',shot)
        
        if print_charts:
            heatmap_with_red_box(current_board_int,shot)

    elif mode == 'hunt':
        
        # Run the hunt algorithm
        plot, shot = hunt(guess_board, ship_list)

        # Print cooridinates and heatmap. 
        print('hunt mode, shot coordinates here: ', shot)
        if print_charts:
            heatmap_with_red_box(plot, shot)
    
    else:
        print('Error in get_shot')
    return shot


### Print the board in a nicer way
def print_board(board_array):
  print("\n   " + "  ".join(str(x) for x in range(0, col_size)))
  for r in range(row_size):
    print(str(r) + "  " + "  ".join(str(c) for c in board_array[r]))
  print()

# Single Run of the Game

In [ ]:
# Time it
start_time = time.time()

#Initialize the board of a row_size by col_size matrix. Also get a guess board
game_board = [[0] * col_size for x in range(row_size)]
guess_board = [[0] * col_size for x in range(row_size)]
#List of ship details
ship_list = []

###Place all the battle ships
placed = 0
while placed < len(ships):
    ship_info = random_location(ships[placed][1],ships[placed][0],game_board, row_size, col_size)
    print(ship_info)
    if ship_info == 'None':
        continue
    else:
        ship_list.append(Ship(ship_info['location'], ship_info['size'], ship_info['name'], ship_info['orientation'], game_board))
    placed += 1

for turn in range(max_turns):
  print('turn number: ',turn)
  guess_coords = {}
  while True:
    shot = get_shot(guess_board, ship_list, print_charts=True)
    guess_coords['row'] = shot[0]
    guess_coords['col'] = shot[1]
    if guess_board[guess_coords['row']][guess_coords['col']] == 'X' or \
     guess_board[guess_coords['row']][guess_coords['col']] == 'M' or \
     guess_board[guess_coords['row']][guess_coords['col']] == 'S':
      print("\nYou guessed that one already.")
    else:
      break

  ship_hit = False
  for ship in ship_list:
    if ship.contains(guess_coords):
      ship_hit = True
      guess_board[guess_coords['row']][guess_coords['col']] = 'X'
      if ship.destroyed(guess_board):
        print("Ship Destroyed!")
        ship_list.remove(ship)
      break
  if not ship_hit:
    guess_board[guess_coords['row']][guess_coords['col']] = 'M'
  
  if not ship_list:
    break

# End Game
if ship_list:
  print("You lose!")
else:
  print("All the ships are sunk. You win!")
  print(turn)
    
#End time and total time
end_time = time.time()
execution_time = end_time - start_time
print(f'Full game took: {execution_time:.5f} seconds')

# Loop Run of the Game

In [ ]:
#set an empty list of results
sim_results = []
number_of_sims = 10000

for sim in range(0,number_of_sims):
    print('-'*15,'Sim Number: ',sim,'-'*15)
    #Initialize the board of a row_size by col_size matrix. Also get a guess board
    game_board = [[0] * col_size for x in range(row_size)]
    guess_board = [[0] * col_size for x in range(row_size)]
    #List of ship details
    ship_list = []

    ###Place all the battle ships
    placed = 0
    while placed < len(ships):
        ship_info = random_location(ships[placed][1],ships[placed][0],game_board, row_size, col_size)
        print(ship_info)
        if ship_info == 'None':
            continue
        else:
            ship_list.append(Ship(ship_info['location'], ship_info['size'], ship_info['name'], ship_info['orientation'], game_board))
        placed += 1

    for turn in range(max_turns):
      print('turn number: ',turn)
      guess_coords = {}
      while True:
        shot = get_shot(guess_board, ship_list, print_charts=False)
        guess_coords['row'] = shot[0]
        guess_coords['col'] = shot[1]
        if guess_board[guess_coords['row']][guess_coords['col']] == 'X' or \
         guess_board[guess_coords['row']][guess_coords['col']] == 'M' or \
         guess_board[guess_coords['row']][guess_coords['col']] == 'S':
          print("\nYou guessed that one already.")
        else:
          break

      ship_hit = False
      for ship in ship_list:
        if ship.contains(guess_coords):
          ship_hit = True
          guess_board[guess_coords['row']][guess_coords['col']] = 'X'
          if ship.destroyed(guess_board):
            print("Ship Destroyed!")
            ship_list.remove(ship)
          break
      if not ship_hit:
        guess_board[guess_coords['row']][guess_coords['col']] = 'M'

      if not ship_list:
        break

    # End Game
    if ship_list:
      print("You lose!")
    else:
      print("All the ships are sunk. You win!")
      print(turn)
    sim_results.append(turn)

# Simple Summary Statistics on the Simulations

In [ ]:
print('mean: ',sum(sim_results)/len(sim_results))
print('median: ',statistics.median(sim_results))

# Calculate the cumulative percentage of observations below each value on the x-axis
x_values = np.linspace(0, 100, 100)
cumulative_percentage = [np.sum(np.array(sim_results) < x) / len(sim_results) * 100 for x in x_values]

# Plot the cumulative percentage
plt.plot(x_values, cumulative_percentage, marker='o')
plt.xlabel('Value on X-axis')
plt.ylabel('Cumulative Percentage below Value')
plt.title('Cumulative Percentage of Observations below Each Value')
plt.grid(True)
plt.show()

# Compute histogram
hist, bins = np.histogram(sim_results, bins=20, density=True)

# Compute midpoints of bins
bin_midpoints = (bins[1:] + bins[:-1]) / 2

# Plot histogram with a line
plt.hist(sim_results, bins=30, density=True, alpha=0.7, color='steelblue', edgecolor='black')

# Overlay a line on the histogram
plt.plot(bin_midpoints, hist, 'k-', linewidth=2)

# Add labels and title
plt.xlabel('Value')
plt.ylabel('Probability Density')
plt.title('Histogram with Line')

# Show the plot
plt.show()


# Create a Manual Game That Takes CSV Input to Run in Real Time

In [ ]:
# Time it
start_time = time.time()

# Get the current working directory
current_directory = os.getcwd()

# Setup blank board to place ships (needed for my code)
game_board = [[0] * col_size for x in range(row_size)]
# Pull in a csv that we'll use as the guess board
csv_file_path = os.path.abspath(os.path.join(current_directory, 'BattleShipBoard.csv'))
df = pd.read_csv(csv_file_path,header=None)
guess_board = np.array(df)

# Make sure 0's are integer and not string
for i in range(len(guess_board)):
        for j in range(len(guess_board[i])):
            if isinstance(guess_board[i][j], str):
                try:
                    guess_board[i][j] = int(guess_board[i][j])
                except ValueError:
                    print(f"Cannot convert {guess_board[i][j]} to integer.")



#Have to put ships on the board
###Place all the battle ships
#List of ship details
ship_list = []
remaining = [('Aircraft Carrier',5),
             ('Battleship',4),
             ('Cruiser',3),
             ('Sub',3),
             ('Destroyer',2)]
placed = 0

while placed < len(remaining):
    ship_info = random_location(remaining[placed][1],remaining[placed][0],game_board, row_size, col_size)
    print(ship_info)
    if ship_info == 'None':
        continue
    else:
        ship_list.append(Ship(ship_info['location'], ship_info['size'], ship_info['name'], ship_info['orientation'], game_board))
    placed += 1


get_shot(guess_board, ship_list, print_charts = True)

#End time and total time
end_time = time.time()
execution_time = end_time - start_time
print(f'Shot algo took: {execution_time:.5f} seconds')